In [1]:
# Short Script to scrape the actions plans correctly 

In [1]:
import urllib.request, urllib.parse, urllib.error, json, re, random
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import datetime as dt
from tqdm import tqdm

In [2]:
# function that takes in a url and returns the html soup

THROTTLE_TIME = 0.2

def getPage(link):
    time.sleep(THROTTLE_TIME * (random.random() * 2))
    result = ''
    while result == '':
        try:
            result = requests.get(url)
        except:
            print("Connection refused by the server, sleeping for 15 seconds...")
            time.sleep(15)
            print("Continuing...")
            continue
    if result.status_code == 200:
        response = result.content.decode('utf-8')
        response = BeautifulSoup(response, "html.parser")
        return response
    else:
        print("Request failed")
        return None

In [3]:
# Initialize dataframe to store data
action_plans = pd.DataFrame()
skipped_cities = pd.DataFrame()

In [4]:
# Get list of cities
url = 'https://carbonn.org/entities?page='
p = BeautifulSoup(requests.get(url + '1').content, 'lxml')
n_pages = p.find('span', {'class': 'results-counter'}).text.replace(' registered entities', '')
n_pages = int(n_pages) // 20

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
cities = []
for i in tqdm(range(1,n_pages+1)):
    p = requests.get(url + str(i)).content
    p = BeautifulSoup(p, 'lxml')
    for lf in p.find_all('div', {'class': 'list-frame'}):
        temp = {}
        name = lf.find('a', {'class': 'open'}).text.split(', ')
        temp['country'] = name[-1]
        temp['name'] = ', '.join(name)
        temp['city_profile_url'] = 'https://carbonn.org' + lf.find('a', {'class': 'btn'})['href']
        cities.append(temp)

In [ ]:
cities = pd.DataFrame(cities)
# cities.to_csv('../../../../output/actors/carbonn/2019carbonn_cities.csv')

In [ ]:
# This loop scrapes the action plans correctly, but takes awhile. going to run it when I have more time
for city, row in tqdm(cities.iterrows()):
    name = row['name']
    country = row['country']
    url = row['city_profile_url']
    url = url.split('&text')[0]
    soup = getPage(url)
    if soup == None:
        holder={}
        holder['city']=name
        holder['url']=url
        skipped_cities = skipped_cities.append(holder, ignore_index=True)
        continue
    holder = {}
    # action plans
    results = soup.find('div', id='tab5')
    if results is not None:
        plans = results.find_all('div', class_='frame')
        for plan in plans:
            holder['city'] = name
            holder['country'] = country
            plan_text = plan.find('a', class_='opener').get_text()
            plan_text = plan_text.replace('\n', '')
            plan_text = plan_text.strip()
            holder['plan_name'] = plan_text
            text= plan.li.get_text()
            year = text.split(': ')[1]
            holder['start year'] = year
            text2= plan.li.next_sibling.next_sibling.get_text()
            type_ = text2.split(': ')[1]
            holder['plan_type'] = type_
            action_plans = action_plans.append(holder, ignore_index=True)

In [ ]:
date = dt.datetime.today().strftime("%m.%d.%y")
action_plans.to_csv('../../../../output/actors/carbonn/'+date+'carbonn_action_plans.csv', encoding = "utf-8")